<a href="https://colab.research.google.com/github/Prkhar05/Tiny_Text2SQL/blob/main/sql_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# downloading

In [ ]:
import gdown

In [ ]:
# https://drive.google.com/file/d/1mkCx2GOFIqNesD4y8TDAO1yX1QZORP5w/view?usp=sharing  #test suite dataset
# https://drive.usercontent.google.com/open?id=1iRDVHLr4mX2wQKSgA9J8Pire73Jahh0m&authuser=0 #spider dataset
# https://drive.usercontent.google.com/open?id=1Y3ydpFiQQ3FC0bzdfy3groV95O_f1nXF&authuser=0 #cosql dataset
# https://drive.usercontent.google.com/open?id=1Uu7NMHTR1tdQw1t7bAuM7OPU4LElVKfg&authuser=0 #sparc dataset

In [ ]:
def download_zip(file_id,output_file):
    gdown.download(f'https://drive.google.com/uc?id={file_id}', output_file, quiet=True)

In [ ]:
# download_zip('1Y3ydpFiQQ3FC0bzdfy3groV95O_f1nXF','cosqldatabases.zip')
# !unzip -q cosqldatabases.zip -d cosqldatabase/

# test-suite-sql-eval

In [ ]:
!pip install -q sqlparse==0.4.2 nltk==3.7

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
download_zip('1mkCx2GOFIqNesD4y8TDAO1yX1QZORP5w','testsuitedatabases.zip')
!unzip -q testsuitedatabases.zip -d database/

In [ ]:
!git clone https://github.com/taoyds/test-suite-sql-eval.git

In [ ]:
!cp -r /content/database/database /content/test-suite-sql-eval

In [ ]:
!cd /content/test-suite-sql-eval && mkdir -p out

In [ ]:
import pickle
with open('/content/test-suite-sql-eval/classical_test.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
question=[]
answer=[]
database=[]
for d in data:
    question.append(d['text'])
    database.append(d['db_id'])
    answer.append(d['query'])
len(question),len(answer),len(database)

(3509, 3509, 3509)

In [ ]:
schema_cache = {}

def get_schema(db):
    if db in schema_cache:
        return schema_cache[db]

    connection = sqlite3.connect(f'/content/database/database/{db}/{db}.sqlite')
    cursor = connection.cursor()

    cursor.execute("SELECT sql FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()
    schema=""
    for table in tables:
        schema+=table[0]+';'
    connection.close()

    schema_cache[db] = schema
    return schema

In [ ]:
database_schema=[]
for db in database:
    schema=get_schema(db)
    database_schema.append(schema)
len(database_schema)

3509

In [ ]:
data = {"schema": database_schema, "question": question, "query": answer}
sql_eval_dataset = pd.DataFrame(data)

In [ ]:
# dataset.iloc[0].schema,dataset.iloc[0].question,dataset.iloc[0].query

In [ ]:
# academic_count = 0
# atis_count = 0
# advising_count = 0
# geography_count = 0
# imdb_count = 0
# restaurants_count = 0
# scholar_count = 0
# yelp_count = 0
# for db in database:
#     if db == "academic":
#         academic_count += 1
#     elif db == "atis":
#         atis_count += 1
#     elif db == "advising":
#         advising_count += 1
#     elif db == "geography":
#         geography_count += 1
#     elif db == "imdb":
#         imdb_count += 1
#     elif db == "restaurants":
#         restaurants_count += 1
#     elif db == "scholar":
#         scholar_count += 1
#     elif db == "yelp":
#         yelp_count += 1
# print("academic:", academic_count)
# print("atis:", atis_count)
# print("advising:", advising_count)
# print("geography:", geography_count)
# print("imdb:", imdb_count)
# print("restaurants:", restaurants_count)
# print("scholar:", scholar_count)
# print("yelp:", yelp_count)
# # ['academic'=196, 'atis'=347, 'advising'=1832, 'geography'=182, 'imdb'=131, 'restaurants'=378, 'scholar'=315, 'yelp'=128]

In [ ]:
# !rm -r /content/test-suite-sql-eval/out/out_academic_test.json

In [ ]:
# !cd test-suite-sql-eval && python3 evaluate_classical.py --gold=classical_test.pkl --pred=evaluation_examples/academic_gold.txt --subset=academic --out_file=out/out_academic_test.json

# sparc

In [ ]:
import json,sqlite3

In [ ]:
download_zip('1Uu7NMHTR1tdQw1t7bAuM7OPU4LElVKfg','sparcdatabases.zip')
!unzip -q sparcdatabases.zip -d sparcdatabase/

In [ ]:
def get_schema_with_sql(db):
    try:
        with open(f"/content/sparcdatabase/sparc/database/{db}/schema.sql", 'r') as sql_file:
            schema = sql_file.read()
    except FileNotFoundError:
        try:
            with open(f"/content/sparcdatabase/sparc/database/{db}/{db}.sql", 'r') as sql_file:
                schema = sql_file.read()
        except FileNotFoundError:
            # print(db)
            schema=""
    return schema

In [ ]:
schema_cache = {}

def get_schema_with_sqlite(db):
    if db in schema_cache:
        return schema_cache[db]

    connection = sqlite3.connect(f'/content/sparcdatabase/sparc/database/{db}/{db}.sqlite')
    cursor = connection.cursor()

    cursor.execute("SELECT sql FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()
    schema=""
    for table in tables:
        schema+=table[0]+';'
    connection.close()

    schema_cache[db] = schema
    return schema

In [ ]:
def prepare_data(path_name):
    with open(f"/content/sparcdatabase/sparc/{path_name}.json", 'r') as f:
        data = f.read()
        data=json.loads(data)
    question=[]
    answer=[]
    database=[]
    for d in data:
        question.append(d['final']['utterance'])
        database.append(d['database_id'])
        answer.append(d['final']['query'])
    database_schema=[]
    for db in database:
        schema=get_schema_with_sql(db)
        database_schema.append(schema)
    dt=['voter_1','world_1','college_2','college_1','small_bank_1','chinook_1','twitter_1','epinions_1','flight_4']
    for i in range(len(database)):
        if database[i] in dt:
            database_schema[i]=get_schema_with_sqlite(database[i])
    return question,answer,database,database_schema

In [ ]:
question,answer,database,database_schema=prepare_data("dev")
len(question),len(answer),len(database),len(database_schema)

In [ ]:
question1,answer1,database1,database_schema1=prepare_data("train")
len(question1),len(answer1),len(database1),len(database_schema1)

In [ ]:
len([d for d in database_schema if d==""])

In [ ]:
database_schema=database_schema+database_schema1
question=question+question1
answer=answer+answer1
data = {"schema": database_schema, "question": question, "query": answer}
sparc_dataset = pd.DataFrame(data)

# spider

In [ ]:
import json,sqlite3

In [ ]:
download_zip('1iRDVHLr4mX2wQKSgA9J8Pire73Jahh0m','spiderdatabases.zip')
!unzip -q spiderdatabases.zip -d spiderdatabase/

In [ ]:
def get_schema_with_sql(db):
    try:
        with open(f"/content/spiderdatabase/spider/database/{db}/schema.sql", 'r') as sql_file:
            schema = sql_file.read()
    except FileNotFoundError:
        try:
            with open(f"/content/spiderdatabase/spider/database/{db}/{db}.sql", 'r') as sql_file:
                schema = sql_file.read()
        except FileNotFoundError:
            # print(db)
            schema=""
    return schema

In [ ]:
schema_cache = {}

def get_schema_with_sqlite(db):
    if db in schema_cache:
        return schema_cache[db]

    connection = sqlite3.connect(f'/content/spiderdatabase/spider/database/{db}/{db}.sqlite')
    cursor = connection.cursor()

    cursor.execute("SELECT sql FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()
    schema=""
    for table in tables:
        schema+=table[0]+';'
    connection.close()

    schema_cache[db] = schema
    return schema

In [ ]:
def prepare_data(path_name):
    with open(f"/content/spiderdatabase/spider/{path_name}.json", 'r') as f:
        data = f.read()
        data=json.loads(data)
    question=[]
    answer=[]
    database=[]
    for d in data:
        question.append(d['question'])
        database.append(d['db_id'])
        answer.append(d['query'])
    database_schema=[]
    for db in database:
        schema=get_schema_with_sql(db)
        database_schema.append(schema)
    dt=['voter_1','world_1','college_2','college_1','small_bank_1','chinook_1','twitter_1','epinions_1','flight_4']
    for i in range(len(database)):
        if database[i] in dt:
            database_schema[i]=get_schema_with_sqlite(database[i])
    return question,answer,database,database_schema

In [ ]:
question,answer,database,database_schema=prepare_data("dev")
len(question),len(answer),len(database),len(database_schema)

In [ ]:
# question1,answer1,database1,database_schema1=prepare_data("train_spider")
# len(question1),len(answer1),len(database1),len(database_schema1)

In [ ]:
question2,answer2,database2,database_schema2=prepare_data("train_others")
len(question2),len(answer2),len(database2),len(database_schema2)

In [ ]:
def get_schema_with_sql(db):
    try:
        with open(f"/content/spiderdatabase/spider/test_database/{db}/schema.sql", 'r') as sql_file:
            schema = sql_file.read()
    except FileNotFoundError:
        try:
            with open(f"/content/spiderdatabase/spider/test_database/{db}/{db}.sql", 'r') as sql_file:
                schema = sql_file.read()
        except FileNotFoundError:
            # print(db)
            schema=""
    return schema

In [ ]:
def prepare_data(path_name):
    with open(f"/content/spiderdatabase/spider/test_data/{path_name}.json", 'r') as f:
        data = f.read()
        data=json.loads(data)
    question=[]
    answer=[]
    database=[]
    for d in data:
        question.append(d['question'])
        database.append(d['db_id'])
        answer.append(d['query'])
    database_schema=[]
    for db in database:
        schema=get_schema_with_sql(db)
        database_schema.append(schema)
    dt=['voter_1','world_1','college_2','college_1','small_bank_1','chinook_1','twitter_1','epinions_1','flight_4']
    for i in range(len(database)):
        if database[i] in dt:
            database_schema[i]=get_schema_with_sqlite(database[i])
    return question,answer,database,database_schema

In [ ]:
question3,answer3,database3,database_schema3=prepare_data("dev")
len(question3),len(answer3),len(database3),len(database_schema3)

In [ ]:
scm="""CREATE TABLE Artists (
  artistID INTEGER,
  lname TEXT,
  fname TEXT,
  birthYear INTEGER,
  deathYear INTEGER,
  PRIMARY KEY (artistID)
);CREATE TABLE Paintings (
  paintingID INTEGER,
  title TEXT,
  year INTEGER,
  height_mm INTEGER,
  width_mm INTEGER,
  medium TEXT,
  mediumOn TEXT,
  location TEXT,
  painterID INTEGER,
  PRIMARY KEY (paintingID),
  FOREIGN KEY (painterID) REFERENCES Artists (artistID)
);CREATE TABLE Sculptures (
  sculptureID INTEGER,
  title TEXT,
  year INTEGER,
  medium TEXT,
  location TEXT,
  sculptorID INTEGER,
  PRIMARY KEY (sculptureID),
  FOREIGN KEY (sculptorID) REFERENCES Artists (artistID)
);"""
for i in range(len(database3)):
    if database3[i]=="art_1":
        database_schema3[i]=scm

In [ ]:
len([d for d in database_schema3 if d==""])

In [ ]:
len(answer)+len(answer2)+len(answer3)

In [ ]:
database_schema=database_schema+database_schema2+database_schema3
question=question+question2+question3
answer=answer+answer2+answer3
len(answer)
data = {"schema": database_schema, "question": question, "query": answer}
spider_dataset = pd.DataFrame(data)

# final

In [ ]:
sql_eval_dataset.shape,spider_dataset.shape,sparc_dataset.shape

In [ ]:
# final_dataset=pd.concat([sql_eval_dataset, spider_dataset, sparc_dataset], axis=0).reset_index(drop=True, inplace=True)
# final_dataset.shape

# pushing dataset

In [ ]:
!pip install -q datasets

In [ ]:
!huggingface-cli login

In [ ]:
from datasets import Dataset,load_dataset,DatasetDict

In [ ]:
dataset=load_dataset('Cynaptics/Test2Sql')
dataset

In [ ]:
d={
    "train":dataset['train'],
    "small_train":dataset['small_train'],
    "small_validation":dataset['small_validation'],
    "sql_eval_test":Dataset.from_pandas(sql_eval_dataset),
    "spider_test":Dataset.from_pandas(spider_dataset),
    "sparc_test":Dataset.from_pandas(sparc_dataset)
   }

In [ ]:
dataset = Dataset.from_dict(d)
dataset

In [ ]:
dataset.push_to_hub("Cynaptics/Test2Sql")